In [1]:
import warnings
import argparse
import DrawingWithTensors
import pandas as pd
from random import randint
import torch
import os
import argparse
import time

# initiate the parser
parser = argparse.ArgumentParser(description = "List of options to run application when creating custom datset")

parser = argparse.ArgumentParser()  
parser.add_argument("-V", "--version", help="show program version", action="store_true")
parser.add_argument("-s", "--size", help="upper bound of number of images to include")
parser.add_argument("-t", "--rotations", help="allow random image and tensor rotations to be made")
parser.add_argument("-f", "--root_folder", help="destination for root folder")
parser.add_argument("-g", "--generation", help="which generation number we are using")
parser.add_argument("-d", "--dimensions", help="square image dimensions")
parser.add_argument("-y", "--scale", help="feet/pixel ratio")

#defined defaults
upper_bound = 10
isize = 416
iteration = "0"
rotations= False
ROOT = "/home/peo5032/Documents/COMP594/input/gen"
factor = 0.45

# read arguments from the command line
args = parser.parse_args(['-g','10',
                          '-s','70000',
                          '-d','416',
                          '-t','true',
                          '-f', '/home/peo5032/Documents/COMP594/input/gen',
                          '-y', '0.05'
                         ])

# check for --version or -V
if args.version:  
    print("this is version 0.1", flush=True)
    
if args.size: 
    print("will create", upper_bound, "images", flush=True)
    upper_bound = int(args.size) + 1

if args.rotations: 
    rotations= args.rotations.lower() == "true"
    print("rotations was", args.rotations, flush=True)
    
if args.root_folder:  
    os.makedirs(args.root_folder, exist_ok=True)
    print("destination was", args.root_folder, flush=True)
    ROOT = args.root_folder
    
if args.generation:
    print("iteration was", args.generation, flush=True)
    iteration = args.generation
    
if args.dimensions:
    print("dimension chosen was", args.dimensions,flush=True)
    isize = int(args.dimensions)
    
if args.scale:
    print("scale ratio chosen was", args.scale,flush=True)
    factor = args.scale
    

os.makedirs(ROOT, exist_ok=True)
IMAGE_PATH = ROOT + iteration + "/roads"
os.makedirs(IMAGE_PATH, exist_ok=True)

TENSOR_PATH = ROOT + iteration + "/tensor_values"
os.makedirs(TENSOR_PATH, exist_ok=True)

PICKLE_PATH = ROOT + iteration

df = pd.DataFrame()
NumLanes = []
ShldrWidth = []
ShldrWidthCenter = []
RoadWidth = []
FileNames = []
imageGen = DrawingWithTensors.datasetFactory(IMAGE_SIZE = isize)
b_type = ["trees", "water", "desert"]
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


will create 10 images
rotations was true
destination was /home/peo5032/Documents/COMP594/input/gen
iteration was 10
dimension chosen was 416
scale ratio chosen was 0.05


In [2]:
tmp_tensor = torch.zeros(1,isize,isize,dtype=torch.float32)
since = time.time()
for i in range(0,upper_bound):
    
    if i % 10 == 0:
        print("Pic {:0d}/{:0d}".format(i,upper_bound), flush=True)
    c = randint(0,80)
    lanecount = randint(1,3)
    laneWidth = randint(17,40)
    lineWidth = randint(1,2)
    shoulderWidth = randint(0,79)
    
    #create tuple of information, img, and tensor
    tuple,img,tmp_tensor = imageGen.generateNewImageWithTensor(c,
                                                               lanecount,
                                                               laneWidth,
                                                               lineWidth,
                                                               shoulderWidth,
                                                               tmp_tensor,
                                                               b_type[randint(0,len(b_type)-1)],
                                                               factor)
    
    roadWidth,laneCount,shoulderWidth,centerShldrWidth = tuple       
    
    if rotations:
        img, tmp_tensor = DrawingWithTensors.rotationOfImageAndTensor(img,
                                                                      tmp_tensor,
                                                                      imageGen.classList,
                                                                      randint(0,180),
                                                                     )
        
    NumLanes.append(laneCount)
    ShldrWidth.append(shoulderWidth)
    RoadWidth.append (roadWidth)
    ShldrWidthCenter.append(centerShldrWidth)
    
    FileName = str(i) + ".png"
    FileNames.append(FileName)
    img.save(IMAGE_PATH + "/" + FileName,"PNG")
    
    #save tensor
    torch.save(tmp_tensor, TENSOR_PATH + "/"+ str(i) + '.pt')
    
    img.close()
    tmp_tensor = torch.mul(tmp_tensor, 0)

df['NumLanes'] = NumLanes
df['ShldrWidth'] = ShldrWidth
df['RdwyWidth'] = RoadWidth
df['ShldrWidthCenter'] = ShldrWidthCenter
df['FileName'] = FileNames

df.to_pickle(PICKLE_PATH + "/train_images_v2.pkl")

time_elapsed = time.time() - since
print('Generation complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)

Pic 0/70001
Pic 10/70001
Pic 20/70001
Pic 30/70001
Pic 40/70001
Pic 50/70001
Pic 60/70001
Pic 70/70001
Pic 80/70001
Pic 90/70001
Pic 100/70001
Pic 110/70001
Pic 120/70001
Pic 130/70001
Pic 140/70001
Pic 150/70001
Pic 160/70001
Pic 170/70001
Pic 180/70001
Pic 190/70001
Pic 200/70001
Pic 210/70001
Pic 220/70001
Pic 230/70001
Pic 240/70001
Pic 250/70001
Pic 260/70001
Pic 270/70001
Pic 280/70001
Pic 290/70001
Pic 300/70001
Pic 310/70001
Pic 320/70001
Pic 330/70001
Pic 340/70001
Pic 350/70001
Pic 360/70001
Pic 370/70001
Pic 380/70001
Pic 390/70001
Pic 400/70001
Pic 410/70001
Pic 420/70001
Pic 430/70001
Pic 440/70001
Pic 450/70001
Pic 460/70001
Pic 470/70001
Pic 480/70001
Pic 490/70001
Pic 500/70001
Pic 510/70001
Pic 520/70001
Pic 530/70001
Pic 540/70001
Pic 550/70001
Pic 560/70001
Pic 570/70001
Pic 580/70001
Pic 590/70001
Pic 600/70001
Pic 610/70001
Pic 620/70001
Pic 630/70001
Pic 640/70001
Pic 650/70001
Pic 660/70001
Pic 670/70001
Pic 680/70001
Pic 690/70001
Pic 700/70001
Pic 710/70001
Pic

Pic 5540/70001
Pic 5550/70001
Pic 5560/70001
Pic 5570/70001
Pic 5580/70001
Pic 5590/70001
Pic 5600/70001
Pic 5610/70001
Pic 5620/70001
Pic 5630/70001
Pic 5640/70001
Pic 5650/70001
Pic 5660/70001
Pic 5670/70001
Pic 5680/70001
Pic 5690/70001
Pic 5700/70001
Pic 5710/70001
Pic 5720/70001
Pic 5730/70001
Pic 5740/70001
Pic 5750/70001
Pic 5760/70001
Pic 5770/70001
Pic 5780/70001
Pic 5790/70001
Pic 5800/70001
Pic 5810/70001
Pic 5820/70001
Pic 5830/70001
Pic 5840/70001
Pic 5850/70001
Pic 5860/70001
Pic 5870/70001
Pic 5880/70001
Pic 5890/70001
Pic 5900/70001
Pic 5910/70001
Pic 5920/70001
Pic 5930/70001
Pic 5940/70001
Pic 5950/70001
Pic 5960/70001
Pic 5970/70001
Pic 5980/70001
Pic 5990/70001
Pic 6000/70001
Pic 6010/70001
Pic 6020/70001
Pic 6030/70001
Pic 6040/70001
Pic 6050/70001
Pic 6060/70001
Pic 6070/70001
Pic 6080/70001
Pic 6090/70001
Pic 6100/70001
Pic 6110/70001
Pic 6120/70001
Pic 6130/70001
Pic 6140/70001
Pic 6150/70001
Pic 6160/70001
Pic 6170/70001
Pic 6180/70001
Pic 6190/70001
Pic 6200/7

Pic 10940/70001
Pic 10950/70001
Pic 10960/70001
Pic 10970/70001
Pic 10980/70001
Pic 10990/70001
Pic 11000/70001
Pic 11010/70001
Pic 11020/70001
Pic 11030/70001
Pic 11040/70001
Pic 11050/70001
Pic 11060/70001
Pic 11070/70001
Pic 11080/70001
Pic 11090/70001
Pic 11100/70001
Pic 11110/70001
Pic 11120/70001
Pic 11130/70001
Pic 11140/70001
Pic 11150/70001
Pic 11160/70001
Pic 11170/70001
Pic 11180/70001
Pic 11190/70001
Pic 11200/70001
Pic 11210/70001
Pic 11220/70001
Pic 11230/70001
Pic 11240/70001
Pic 11250/70001
Pic 11260/70001
Pic 11270/70001
Pic 11280/70001
Pic 11290/70001
Pic 11300/70001
Pic 11310/70001
Pic 11320/70001
Pic 11330/70001
Pic 11340/70001
Pic 11350/70001
Pic 11360/70001
Pic 11370/70001
Pic 11380/70001
Pic 11390/70001
Pic 11400/70001
Pic 11410/70001
Pic 11420/70001
Pic 11430/70001
Pic 11440/70001
Pic 11450/70001
Pic 11460/70001
Pic 11470/70001
Pic 11480/70001
Pic 11490/70001
Pic 11500/70001
Pic 11510/70001
Pic 11520/70001
Pic 11530/70001
Pic 11540/70001
Pic 11550/70001
Pic 1156

KeyboardInterrupt: 

In [ ]:
from PIL import Image
#testing output of data-set
img = Image.open(IMAGE_PATH+"/7.png")
test_tensor = torch.load(TENSOR_PATH+'/7.pt')
img

In [ ]:
DrawingWithTensors.showInferenceOnImage(img, test_tensor, "road", 0.6, imageGen.classMap)